In [1]:
!pip uninstall pandas
!pip install pandas==1.1.5

Uninstalling pandas-1.2.5:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/pandas-1.2.5.dist-info/*
    /usr/local/lib/python3.7/dist-packages/pandas/*
Proceed (y/n)? n
  Using cached https://files.pythonhosted.org/packages/fd/70/e8eee0cbddf926bf51958c7d6a86bc69167c300fa2ba8e592330a2377d1b/pandas-1.1.5-cp37-cp37m-manylinux1_x86_64.whl
ERROR: pyldavis 3.3.1 has requirement pandas>=1.2.0, but you'll have pandas 1.1.5 which is incompatible.
  Found existing installation: pandas 1.2.5
    Uninstalling pandas-1.2.5:
      Successfully uninstalled pandas-1.2.5


In [2]:
!pip install pyLDAvis 

  Using cached https://files.pythonhosted.org/packages/e6/0a/90da8840e044c329a0271fb0244ff40a68a2615bc360c296a3dc5e326ab6/pandas-1.2.5-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.5 which is incompatible.
  Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5


In [3]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import nltk
import seaborn as sns
from wordcloud import STOPWORDS,WordCloud
from nltk.stem import WordNetLemmatizer 
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
from nltk import corpus
from nltk import wordnet
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import string
import os
import re
import pickle 
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [4]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
#data loader class
class CsvLoader:
  def __init__(self,path,filename):
    if os.getcwd() != path:
      os.chdir(path)   
    self.path = path
    self.filename = filename 
 
  def load_csv(self):
    df = pd.read_csv(self.filename,engine='python',error_bad_lines=False)
    return df
  

In [6]:
csv_loader = CsvLoader('/content','processed_tweet_data.csv')
csv_loaded = csv_loader.load_csv()
csv_loaded.dropna(subset = ["original_text"], inplace=True)
print(csv_loaded)

                     created_at  ...                                  user_mentions
0     2021-06-18 17:55:49+00:00  ...                 ['TelGlobalHealth', 'WHOAFRO']
1     2021-06-18 17:55:59+00:00  ...                             ['globalhlthtwit']
2     2021-06-18 17:56:07+00:00  ...  ['NHSRDForum', 'Research2note', 'NHSRDForum']
3     2021-06-18 17:56:10+00:00  ...                               ['HighWireTalk']
4     2021-06-18 17:56:20+00:00  ...                                 ['PeterHotez']
...                         ...  ...                                            ...
3715  2021-06-19 07:40:29+00:00  ...                               ['pseudofijian']
3716  2021-06-19 07:40:29+00:00  ...                                 ['edendomain']
3717  2021-06-19 07:41:00+00:00  ...                            ['N_RamchanderRao']
3718  2021-06-19 07:41:26+00:00  ...                               ['HighWireTalk']
3719  2021-06-19 07:41:45+00:00  ...                                    ['WH

#Data Understanding
Display information on the Data

In [7]:
class DataInfo:
  def __init__(self,df):
    self.df = df

  def general_info(self):
    return self.df.info()

  def general_description(self):
    return self.df.describe()
  
  def count_null(self):
    return self.df.isnull().sum()

In [8]:
Get_Info = DataInfo(csv_loaded)
print(Get_Info.general_info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3720 entries, 0 to 3719
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   created_at       3720 non-null   object 
 1   source           3720 non-null   object 
 2   original_text    3720 non-null   object 
 3   polarity         3720 non-null   float64
 4   subjectivity     3720 non-null   float64
 5   lang             3720 non-null   object 
 6   favorite_count   3720 non-null   float64
 7   retweet_count    3720 non-null   float64
 8   original_author  3720 non-null   object 
 9   followers_count  3720 non-null   int64  
 10  friends_count    3720 non-null   int64  
 11  user_mentions    3720 non-null   object 
dtypes: float64(4), int64(2), object(6)
memory usage: 377.8+ KB
None


In [9]:
print(Get_Info.general_description())

          polarity  subjectivity  ...  followers_count  friends_count
count  3720.000000   3720.000000  ...     3.720000e+03    3720.000000
mean      0.145035      0.445425  ...     9.375893e+03    2212.945968
std       0.210508      0.218848  ...     2.291726e+05    6351.014588
min      -1.000000      0.000000  ...     0.000000e+00       0.000000
25%       0.000000      0.288221  ...     1.437500e+02     284.500000
50%       0.136364      0.450000  ...     5.940000e+02     834.500000
75%       0.278571      0.613636  ...     2.103750e+03    2418.000000
max       1.000000      1.000000  ...     1.366658e+07  237298.000000

[8 rows x 6 columns]


In [10]:
print(Get_Info.count_null())

created_at         0
source             0
original_text      0
polarity           0
subjectivity       0
lang               0
favorite_count     0
retweet_count      0
original_author    0
followers_count    0
friends_count      0
user_mentions      0
dtype: int64


In [11]:
import pandas as pd


class Clean_Tweets:
    """
    The PEP8 Standard AMAZING!!!
    """
    def __init__(self, df:pd.DataFrame):
        self.df = df
        print('Automation in Action...!!!')

    def drop_duplicate(self)->pd.DataFrame:
        """
        drop duplicate rows
        """        
        return self.df.drop_duplicates()

    def convert_to_datetime(self,df)->pd.DataFrame:
        """
        convert column to datetime
        """
        df['created_at'] = pd.to_datetime(self.df['created_at'])
        return df
    
    def convert_to_numbers(self, df:pd.DataFrame)->pd.DataFrame:
        """
        convert columns like polarity, subjectivity, retweet_count
        favorite_count etc to numbers
        """
        df['polarity'] = pd.to_numeric(self.df['polarity'])
        df['subjectivity'] = pd.to_numeric(self.df['subjectivity'])
        df['retweet_count'] = pd.to_numeric(self.df['retweet_count'])
        df['favorite_count'] = pd.to_numeric(self.df['favorite_count'])
        df['followers_count'] = pd.to_numeric(self.df['followers_count'])
        return df

    def drop_null_col(self, df:pd.DataFrame)->pd.DataFrame:
        # removed columns containing null values of more than 20%
        row,col = df.shape
        df.dropna(axis='columns',thresh=row*0.8,inplace=True)
        print(df.columns)
        return df
        

    def special_chars(self,x):
        special_characters = '@_!#$%^&*()<>?/\|}{~:;[]'
        for char in special_characters:
            x = x.replace(char, '')
        x = x.encode('ascii', 'ignore').decode('ascii')
        return x

    def filter_text(self,df:pd.DataFrame) ->pd.DataFrame:
        df['clean_text'] = self.df['original_text'].apply(lambda x: self.special_chars(x))
        # print(df['original_text'])
        return df

    def clean_dataframe(self):
        df_duplicatedropped = self.drop_duplicate()
        df_datetime = self.convert_to_datetime(df_duplicatedropped)
        df_numb =  self.convert_to_numbers(df_datetime)
        df_nulldroped = self.drop_null_col(df_numb)
        df_cleaned = self.filter_text(df_nulldroped)
        return df_cleaned
    

<input>:46: DeprecationWarning: invalid escape sequence \|
<input>:46: DeprecationWarning: invalid escape sequence \|
<input>:46: DeprecationWarning: invalid escape sequence \|
<input>:46: DeprecationWarning: invalid escape sequence \|
<input>:46: DeprecationWarning: invalid escape sequence \|
<input>:46: DeprecationWarning: invalid escape sequence \|
<input>:46: DeprecationWarning: invalid escape sequence \|
<input>:46: DeprecationWarning: invalid escape sequence \|
<input>:46: DeprecationWarning: invalid escape sequence \|
<input>:46: DeprecationWarning: invalid escape sequence \|
<input>:46: DeprecationWarning: invalid escape sequence \|
<input>:46: DeprecationWarning: invalid escape sequence \|
<input>:46: DeprecationWarning: invalid escape sequence \|
<input>:46: DeprecationWarning: invalid escape sequence \|
<input>:46: DeprecationWarning: invalid escape sequence \|
<input>:46: DeprecationWarning: invalid escape sequence \|
<ipython-input-11-2cc2fd0ece4b>:46: DeprecationWarning: 

In [12]:
  cleanData = Clean_Tweets(csv_loaded).clean_dataframe()

Automation in Action...!!!
Index(['created_at', 'source', 'original_text', 'polarity', 'subjectivity',
       'lang', 'favorite_count', 'retweet_count', 'original_author',
       'followers_count', 'friends_count', 'user_mentions'],
      dtype='object')


#Data Preparation
Select and clean the data we are going to use for our model

In [13]:
class PreparedData:
  def __init__(self,df):
    self.df = df

  def select_data(self):
    df_en = self.df[self.df.lang == "en"]
    sel_df = df_en['original_text']
    sentence_list = sel_df.tolist()
    return sentence_list

  def tokenize_data(self,sentence_list):
    #creating sentences and words from the input tweets
    words = []
    for sent in sentence_list:
      for myWord in sent.split():
        words.append(myWord.lower())   
    return words 

  def lemmatize_data(self,word):
    wl = WordNetLemmatizer()
    return str(wl.lemmatize(word))

  def remove_stop_words(self,word):
    s_words = stopwords.words('english')
    if word not in s_words:
      return word

  def remove_none(self,wordlist):
    cleaned_list = []
    for w in wordlist:
      if w is not None:
        cleaned_list.append(w)
    return cleaned_list

  def prepare_data(self):
    # PreparedData_object = PreparedData(self.df)
    prepared_sentencelist = self.select_data()
    word_list = self.tokenize_data(prepared_sentencelist)
    prepared = [self.remove_stop_words(self.lemmatize_data(word)) for word in word_list]
    prepared_wordList = self.remove_none(prepared)
    word_id = corpora.Dictionary([prepared_wordList])
    myCorpus = [word_id.doc2bow(prepared_word) for prepared_word in [prepared_wordList]]
    return prepared_wordList,word_id,myCorpus  
  
  

In [14]:
prepared_wordlist,word_id,myCorpus = PreparedData(csv_loaded).prepare_data()

In [15]:
prepared_wordlist[:10]

['africa',
 '"in',
 'midst',
 'full-blown',
 'third',
 'wave"',
 'coronavirus,',
 'head',
 'whoafro',
 'ha']

#Modeling
Selecting the right model and assessing it 

In [16]:
class Modeling:
  def __init__(self,prepared_wordlist,word_id,myCorpus):
    self.wordList = prepared_wordlist
    self.word_id = word_id
    self.corpus = myCorpus
  
  def LDA_model(self,num_topics = 4,chunksize=100,passes=10,alpha='auto'):
    return gensim.models.ldamodel.LdaModel(self.corpus,
                                           id2word=self.word_id,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=chunksize,
                                           passes=passes,
                                           alpha=alpha,
                                           per_word_topics=True)
    
  def display_info_model(self,model):
    pprint(model.show_topics(formatted=False))


In [17]:
Model = Modeling(prepared_wordlist,word_id,myCorpus)
ldaModel = Model.LDA_model(num_topics = 5,chunksize=80,passes=10,alpha=100)
Model.display_info_model(ldaModel)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

[(0,
  [('vaccine', 0.018008132),
   ('covid19', 0.012284874),
   ('amp', 0.005209174),
   ('need', 0.003983467),
   ('health', 0.003831489),
   ('ha', 0.0036092754),
   ('one', 0.00319636),
   ('case', 0.0030635723),
   ('africa', 0.0026451715),
   ('third', 0.002605841)]),
 (1,
  [('vaccine', 0.009276815),
   ('covid19', 0.0052497704),
   ('amp', 0.0017974391),
   ('case', 0.0017289855),
   ('health', 0.001657009),
   ('need', 0.001645703),
   ('third', 0.0014867457),
   ('ha', 0.0013912953),
   ('one', 0.00129257),
   ('vaccines.', 0.0012797262)]),
 (2,
  [('vaccine', 0.0024109967),
   ('covid19', 0.0023080914),
   ('health', 0.0009942118),
   ('ha', 0.0008574594),
   ('need', 0.0007746557),
   ('amp', 0.0007534756),
   ('dos', 0.00070106785),
   ('africa', 0.00068375305),
   ('case', 0.0006366528),
   ('today', 0.00058584433)]),
 (3,
  [('covid19', 0.005599586),
   ('vaccine', 0.005514818),
   ('amp', 0.0017575963),
   ('case', 0.0014282905),
   ('need', 0.0014253019),
   ('dos', 0

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)


In [18]:
# Evaluation using coherence
class evaluation:
  def __init__(self,model,prepared_wordlist,word_id):
    self.model = model
    self.prepared_wordlist = prepared_wordlist
    self.word_id = word_id

  def coherence_eval(self):
    lda_coherence_mod = CoherenceModel(model=self.model, texts=[self.prepared_wordlist], dictionary=self.word_id, coherence='c_v')
    coherence = lda_coherence_mod.get_coherence()
    print(f'Coherence score and accuracy {coherence}', )

In [19]:
evaluation(ldaModel,prepared_wordlist,word_id).coherence_eval()

Coherence score and accuracy 0.2649209804865912


Visualization

In [20]:
#Visualizing
pyLDAvis.enable_notebook()
LDAvis_prepared = gensimvis.prepare(ldaModel, myCorpus, word_id)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.146021 -0.000001       1        1  99.486803
0     -0.021058  0.000023       2        1   0.158445
1      0.032783 -0.000314       3        1   0.122285
3      0.045062  0.000351       4        1   0.119316
2      0.089235 -0.000059       5        1   0.113151, topic_info=             Term         Freq        Total Category  logprob  loglift
6096      vaccine  3409.000000  3409.000000  Default  30.0000  30.0000
1489      covid19  2618.000000  2618.000000  Default  29.0000  29.0000
674           amp   810.000000   810.000000  Default  28.0000  28.0000
4254         need   684.000000   684.000000  Default  27.0000  27.0000
2543       health   668.000000   668.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
6044     urgently     0.040818   238.278400   Topic5  -7.8547  -1.8879
4854       public     0.041819   274.041157   Topic5  -7.8305  -2.0035
3365         huge     0.040948   246.477854   Topic5  -7.8515  -1.9185
4470  palestinian     0.040553   237.100542   Topic5  -7.8612  -1.8894
3471        india     0.044126   380.033750   Topic5  -7.7768  -2.2768

[362 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
62        1  0.809248     'write
100       1  0.811142     -these
125       1  0.814175       .the
127       1  0.809445       .who
153       1  0.814191  1,882,900
...     ...       ...        ...
6251      1  1.000461       week
6307      1  0.810211      with,
6308      1  0.810233      with.
6335      1  0.998237      world
6338      1  0.999832     world.

[178 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 2, 4, 3])